# 1. Import libraries

In [1]:
import torch
import torch.nn as nn
#from torch.nn.parallel import DistributedDataParallel
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
import shutil
import re
import os
from konlpy.tag import Mecab
from datetime import datetime
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')  #To ignore warnings in ipython.

2021-12-14 19:40:28.440298: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


# 2. Set paths
 - To load dataset
 - To save checkpoints & best checkpoints

In [2]:
#Set directories as you want.
path = "/home/ubuntu/Desktop/Project"
dataset_path = os.path.join(path, "datasets/circlin_feeds_dataset/tokenized_text")

date = datetime.today().strftime("%Y%m%d")
print(f"Date today: {date}")
checkpoint_path = os.path.join(path, f"autolabeler_classifier/bert_model/{date}")
model_path = os.path.join(path, f"autolabeler_classifier/bert_model/{date}")
metric_path = os.path.join(path, f"autolabeler_classifier/bert_model/{date}")

Date today: 20211214


# 3. Training settings

## 3-1. Hyperparameters

In [3]:
# Hyperparameters
MAX_LEN = 256
TRAIN_BATCH_SIZE = 64
VALID_BATCH_SIZE = 64
EPOCHS = 32
LEARNING_RATE = 1e-05 #1e-05 = 0.00001
PRETRAINED = 'bert-base-multilingual-cased'

## 3-2. Loss functions

In [4]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets) #BCEWithLogitsLoss: for Multi-label classification 

## 3-3. Check GPU status & Enable distributed processing
 - __Should be improved!__ 
   - As is : Using DatParallel
   - To be: Use DistributedDataParallel

In [5]:
#Device check(for GPU computing)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [6]:
#For multiple GPU utilization: This should be improved...

# dist.init_process_group(
#     backend='nccl',
#     init_method='tcp://localhost:9999', #FREEPORT
#     world_size=2,
#     rank=0,
# )

# dist.init_process_group(
#     backend="nccl",
#     init_method='tcp://127.0.0.1:9999',
#     rank=0,
#     world_size=2)

## 3-4. Optimizer & BERT tokenizer(from pretrained)

In [7]:
#Optimizer
def make_optimizer(model, lr):
    optimizer = torch.optim.Adam(
        params =  model.parameters(), 
        lr=lr)

    return optimizer

In [8]:
#tokenizer
tokenizer = BertTokenizer.from_pretrained(PRETRAINED, do_lower_case=False)

# 4. Prepare dataset

## 4-1. Define target labels(46)

In [9]:
#Define taret labels
labels = ['간편식', '건강간식', '건강식', '건강음료', '걷기/산책', '격투기', '골프', 
          '기타식단', '기타운동', '농구', '달리기/조깅', '당구', '등산/등반', '루틴기록', '맨몸', '무술', 
          '배구', '배드민턴', '보조제', '보충제', '볼링', '수상스포츠', '스키/스노보드', '승마', '신체기록', 
          '야구', '온라인클래스', '요가', '운동기구', '운동용품', '웨이트', '유산소기록', '의류', '일반간식', 
          '일반식', '일반음료', '일상생활', '자전거', '종합운동', '줄넘기', '축구/풋살', '탁구', '테니스', 
          '폴댄스', '필라테스', '홈트'] #46

## 4-2. Create custom dataset

In [10]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.feed_text = df['text']
        self.labels = self.df[labels].values
        self.max_len = max_len

    def __len__(self):
        return len(self.feed_text)

    def __getitem__(self, index):
        feed_text = str(self.feed_text[index])
        feed_text = " ".join(feed_text.split())

        inputs = self.tokenizer.encode_plus(
            feed_text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.labels[index])
        }

In [11]:
#Get text dataset(tokenized)
dataset = os.path.join(dataset_path, "tokenized_text_dataset(20211125).csv")
whole_df = pd.read_csv(dataset)

In [12]:
# Drop useless features/columns
whole_df.drop(labels=['index', 'seq'], axis=1, inplace=True)

whole_df.head(10)

,text,간편식,건강간식,건강식,건강음료,걷기/산책,격투기,골프,기타식단,기타운동,...,일상생활,자전거,종합운동,줄넘기,축구/풋살,탁구,테니스,폴댄스,필라테스,홈트
0,아침 요거트볼 아침 부터 잠옷 바람 에 민낯 으로 영상 찍 는 용기 는 어플 이 없...,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,러닝 하 고 찍 엇 더니 머리 는 산발 에 눈썹 이 다 지워졌 네요 뎨동해오헬로치즈...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,km 요새 등산 못 갔 더니 체력 이 쓰레기 가 된 거 같 은데 숨 맥혀 다리 가 ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,아침 베 노프 단호박 무화과 오늘 은 조출 이 라 빨리 꺼내 먹 을 수 있 는 걸루...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,클로이팅 힙 하체 클로이팅 복근 싸이클 분 오늘 왜 케 피곤 하 지 내일 체온 당번...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,아침 무화과 오픈 토스트 으으 추워 이제 아침 에 따뜻 한 게 먹 고 싶 군 무화과...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,어깨 클로이팅 복근 주 챌린지 싸이클 분 전신 거울 옮기 기 귀찮 아서 사진 은 패...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7,아침 오 나오,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,등 클로이팅 복근 주 챌린지 싸이클 분 좁아터진 원룸 에서 머리 는 산발 하 고 열...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,아침 요거트볼 주말 내내 누워 만 있 다가 맞이 하 는 월요일 은 너무 피곤 하 다...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
#Train - validation split
train_size = 0.8
splitted_train_df = whole_df.copy().sample(frac=train_size, random_state=200).reset_index(drop=True)
val_df = whole_df.drop(splitted_train_df.index).reset_index(drop=True)

train_dataset = CustomDataset(splitted_train_df, tokenizer, MAX_LEN)
valid_dataset = CustomDataset(val_df, tokenizer, MAX_LEN)

In [14]:
train_data_loader = torch.utils.data.DataLoader(train_dataset, 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(valid_dataset, 
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

# 5. Make feed text classification model

## 5-1. Define functions that save checkpoint of model

In [15]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], valid_loss_min #valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

## 5-2. Define BERT model as a class
 - Use transformer's pretrained model.

In [16]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained(PRETRAINED, return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 46) #(768, len(labels))
    
    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids, 
            attention_mask=attn_mask, 
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

In [17]:
#model
model = BERTClass()
model = model.cuda()
model = nn.DataParallel(model) # Distributed
#model = nn.parallel.DistributedDataParallel(model, device_ids=[0, 1]) #Distributed DataParallel  ===> Should use this!!!!!!!!!!!!!!!!!
model.to(device)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


DataParallel(
  (module): BERTClass(
    (bert_model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(119547, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
          

In [ ]:
# optimizer
optimizer = make_optimizer(model, LEARNING_RATE) #LEARNING_RATE = 1e-05

In [ ]:
val_targets=[]
val_outputs=[]

## 5-3. Training
 - __<u>Add Train Loss!!!!!!!!!!</u>__

In [ ]:
learning_rate = [0.000001, 0.00001, 0.0001, 0.001]
train_losses_lr = {}
avg_train_losses_lr = {}
val_losses_lr = {}
avg_val_losses_lr = {}
epoch_list = [int(x) for x in np.linspace(1, EPOCHS, EPOCHS).tolist()]
print(epoch_list)

In [ ]:
def train_model(n_epochs, 
                        training_loader, 
                        validation_loader, 
                        model, 
                        optimizer, 
                        checkpoint_path, 
                        best_model_path,
                        metric_path,
                        date):
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf
    train_loss_epoch = []
    avg_train_loss_epoch = []
    val_loss_epoch = [] #append to val_loss_list
    avg_val_loss_epoch = [] #append to avg_val_list
    
    for epoch in range(1, n_epochs+1):        
        train_loss = 0
        valid_loss = 0

        model.train()
        print('############# Epoch {}: Training Start   #############'.format(epoch))
        for batch_idx, data in enumerate(training_loader):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)

            outputs = model(ids, mask, token_type_ids)

            optimizer.zero_grad()
            loss = loss_fn(outputs, targets)
            if batch_idx%5000==0:
                print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))   
        print('############# Epoch {}: Training End     #############'.format(epoch))
        train_loss_epoch.append(train_loss)
        print('############# Epoch {}: Validation Start   #############'.format(epoch))
        ######################    
        # validate the model #
        ######################

        model.eval()

        with torch.no_grad():
            for batch_idx, data in enumerate(validation_loader, 0):
                ids = data['input_ids'].to(device, dtype = torch.long)
                mask = data['attention_mask'].to(device, dtype = torch.long)
                token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
                targets = data['targets'].to(device, dtype = torch.float)
                outputs = model(ids, mask, token_type_ids)

                loss = loss_fn(outputs, targets)
                valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
                val_targets.extend(targets.cpu().detach().numpy().tolist())
                val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

            print('############# Epoch {}: Validation End     #############'.format(epoch))
           # calculate average losses
#             print('before calculate avg train loss', train_loss)
            val_loss_epoch.append(valid_loss) 
            avg_train_loss = train_loss/len(training_loader)
            avg_valid_loss = valid_loss/len(validation_loader)
            #Print training/validation statistics
            print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
                epoch, 
                avg_train_loss,
                avg_valid_loss
            ))
            avg_train_loss_epoch.append(avg_train_loss)
            avg_val_loss_epoch.append(avg_valid_loss)              

            # create checkpoint variable and add important data
            checkpoint = {
                    'epoch': epoch + 1,
                    'valid_loss_min': avg_valid_loss,
                    'state_dict': model.state_dict(),
                    'optimizer': optimizer.state_dict()
              }

            save_ckp(checkpoint, False,  f"{checkpoint_path}_{epoch}", best_model_path)

            ## TODO: save the model if validation loss has decreased
            if avg_valid_loss <= valid_loss_min:
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min, avg_valid_loss))
                # save checkpoint as best model
                save_ckp(checkpoint, True,  f"{checkpoint_path}_{epoch}", best_model_path)
                valid_loss_min = avg_valid_loss
        
        now = datetime.today().strftime("%Y-%m-%d %H:%M:%S")
        log_text = f"[{now}]: [Learning Rate {lr}, Epoch {epoch}] - train_loss = {train_loss}, avg_train_loss = {avg_train_loss}, validation_loss = {valid_loss}, avg_validation_loss = {avg_valid_loss}\n"
        if os.path.isfile(os.path.join(metric_path, f"metric_logs_bert_{date}.txt")):
            with open(os.path.join(metric_path, f"metric_logs_bert_{date}.txt"), 'a', encoding='utf-8') as f:
                f.write(log_text)
        else:
            with open(os.path.join(metric_path, f"metric_logs_bert_{date}.txt"), 'w', encoding='utf-8') as f:
                f.write(log_text)
        print('############# Epoch {}  Done   #############\n'.format(epoch))

    train_losses_lr[lr] = train_loss_epoch
    avg_train_losses_lr[lr] = avg_train_loss_epoch
    val_losses_lr[lr] = val_loss_epoch
    avg_val_losses_lr[lr] = avg_val_loss_epoch
    print(f"train_losses_lr for LR {lr}: \n {train_losses_lr}")
    print(f"avg_train_losses_lr for LR {lr}: \n {avg_train_losses_lr}")
    print(f"val_losses_lr for LR {lr}: \n {val_losses_lr}")
    print(f"avg_val_losses_lr {lr}: \n {avg_val_losses_lr}")
    
    return model

### Set checkpoint path, best model's path.

In [ ]:
#ckpt_path = os.path.join(checkpoint_path, "curr_ckpt")
best_model_path = os.path.join(checkpoint_path, "best_model.pt")

### Training start!

In [ ]:
#For hyperparameter tuning
for lr in learning_rate:
    print('\n')    
    print(f'##########################################################')
    print(f'##########################################################')    
    print(f'############### Training for learning rate {lr} START! ###############')
    print(f'##########################################################')
    print(f'##########################################################')
    print('\n')    
    optimizer = make_optimizer(model, lr)
    train_model(EPOCHS,
               train_data_loader,
               val_data_loader,
               model,
               optimizer,
               os.path.join(checkpoint_path, f"curr_ckpt_{lr}"),
               best_model_path,
                metric_path,
               date)

### Check & Visualize validation loss

In [ ]:
color = ['dodgerblue', 'green', 'violet', 'orange']

In [ ]:
plt.figure(figsize=(16, 10))
plt.xticks(epoch_list)
for index, lr in enumerate(learning_rate):
    plt.title("Losses per epoch")
    plt.plot(epoch_list, 
             train_losses_lr[lr], 
             color=color[index], 
             label=f"{lr} - Train")    
    plt.plot(epoch_list, 
             val_losses_lr[lr], 
             '--', 
             color=color[index], 
             label=f"{lr} - Val")
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(loc="upper right")

plt.show()

In [ ]:
plt.figure(figsize=(16, 10))
plt.xticks(epoch_list)
for index, lr in enumerate(learning_rate):
    plt.title("Gap of loss per epoch")
    plt.plot(epoch_list, 
             np.array(train_losses_lr[lr]) - np.array(val_losses_lr[lr]), 
             color=color[index], 
             label=lr)    
    plt.xlabel('Epochs')
    plt.ylabel('Gap of loss')
    plt.legend(loc="upper right")

plt.show()

In [ ]:
plt.figure(figsize=(16, 10))
plt.xticks(epoch_list)
for index, lr in enumerate(learning_rate):
    plt.title("Average losses per epoch")
    plt.plot(epoch_list, 
             avg_train_losses_lr[lr], 
             color=color[index], 
             label=f"{lr} - Train")    
    plt.plot(epoch_list, 
             avg_val_losses_lr[lr], 
             '--', 
             color=color[index], 
             label=f"{lr} - Val")
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(loc="upper right")

plt.show()

In [ ]:
plt.figure(figsize=(16, 10))
plt.xticks(epoch_list)
for index, lr in enumerate(learning_rate):
    plt.title("Gap of Average loss per epoch")
    plt.plot(epoch_list, 
             np.array(avg_train_losses_lr[lr]) - np.array(avg_val_losses_lr[lr]), 
             color=color[index], 
             label=lr)    
    plt.xlabel('Epochs')
    plt.ylabel('Gap of average loss')
    plt.legend(loc="upper right")

plt.show()

- Let's see low losses in detail.

In [ ]:
plt.figure(figsize=(16, 10))
plt.xticks(epoch_list)
for index, lr in enumerate(learning_rate[1:3]):
    plt.title("Losses per epoch: lr = [1e-05, 1e-04]")
    plt.plot(epoch_list, 
             train_losses_lr[lr], 
             color=color[index], 
             label=f"{lr} - Train")    
    plt.plot(epoch_list, 
             val_losses_lr[lr], 
             '--', 
             color=color[index], 
             label=f"{lr} - Val")
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(loc="upper right")

plt.show()

In [ ]:
plt.figure(figsize=(16, 10))
plt.xticks(epoch_list)
for index, lr in enumerate(learning_rate[1:3]):
    plt.title("Gap of loss per epoch: lr = [1e-05, 1e-04]")
    plt.plot(epoch_list, 
             np.array(train_losses_lr[lr]) - np.array(val_losses_lr[lr]), 
             color=color[index], 
             label=lr)    
    plt.xlabel('Epochs')
    plt.ylabel('Gap of loss')
    plt.legend(loc="upper right")

plt.show()

In [ ]:
plt.figure(figsize=(16, 10))
plt.xticks(epoch_list)
for index, lr in enumerate(learning_rate[1:3]):
    plt.title("Average losses per epoch: lr = [1e-05, 1e-04]")
    plt.plot(epoch_list, 
             avg_train_losses_lr[lr], 
             color=color[index], 
             label=f"{lr} - Train")    
    plt.plot(epoch_list, 
             avg_val_losses_lr[lr], 
             '--', 
             color=color[index], 
             label=f"{lr} - Val")
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(loc="upper right")

plt.show()

In [ ]:
plt.figure(figsize=(16, 10))
plt.xticks(epoch_list)
for index, lr in enumerate(learning_rate[1:3]):
    plt.title("Gap of Average loss per epoch: lr = [1e-05, 1e-04]")
    plt.plot(epoch_list, 
             np.array(avg_train_losses_lr[lr]) - np.array(avg_val_losses_lr[lr]), 
             color=color[index], 
             label=lr)    
    plt.xlabel('Epochs')
    plt.ylabel('Gap of average loss')
    plt.legend(loc="upper right")

plt.show()

__Minimum validation loss:  Learning rate = 1e-05, epoch = 17__

## 5-5. Evaluate model
 - Test with validation dataset
 - 1st important index: Precision
 - 2nd importand index: Recall
   - __First, get high & stable <u>Precision</u>, then improve <u>Recall</u>.__
 - And other indexes: F1 score, confusion matrix
 
 - For metrics reference here: https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics

In [19]:
from tqdm.notebook import tqdm
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
#from sklearn.metrics import label_ranking_average_precision_score


def get_metrics(models_path, average_param, label_names):
    #Load model
    load_model = BERTClass()
    load_model = load_model.cuda() #for GPU computation
    load_model = nn.DataParallel(load_model) # Distributed
    
    y_true = valid_dataset.labels
    y_evaluations = {}

    for file in os.listdir(models_path):
        y_pred = []
        
        if file.split('_')[0] == 'curr':           
            model = os.path.join(models_path, file)
            # optimizer
            val_lr = float(file.split('_')[2])
            val_optimizer = make_optimizer(load_model, val_lr) #LEARNING_RATE = 1e-05            
            print(f"===========================Start evaluating model {file}, at lr {val_lr}===========================")
            #Load model --> This isn't necessary if you evaluate model right after training.
            prediction_model = load_ckp(model,
                                                        load_model,
                                                        val_optimizer)[0] #load_ckp: [model, optimizer, checkpoint['epoch'], valid_loss_min.item()]
            
            #Evaluate.
            for i in tqdm(range(len(valid_dataset))):
                prediction = np.array(inference(valid_dataset.feed_text[i],
                                                            prediction_model, 
                                                            tokenizer,
                                                            device))[0]
                prediction_over_threshold = np.where(prediction > 0.1, 1, 0).tolist()  #Threshold for each class is 0.1
                y_pred.append(prediction_over_threshold)
                if i != 0 and i % 5000 == 0:
                    _precision_score = precision_score(y_true[:i+1], y_pred, average=average_param) #average: ['micro', 'macro', 'weighted', 'samples']
                    _recall_score = recall_score(y_true[:i+1], y_pred, average=average_param)
                    _f1_score = f1_score(y_true[:i+1], y_pred, average=average_param)
                    _average_precision_score = average_precision_score(y_true[:i+1], y_pred, average=average_param)
                    print(f'current index number: {i}')
                    print(f'current Precision: {_precision_score}')
                    print(f'current Recall: {_recall_score}')
                    print(f'current f1 score: {_f1_score}')
                    print(f'current AP: {_average_precision_score}')
                    print('\n')
                
                
            #Append evaluation result into y_preds
            _precision_score = precision_score(y_true, y_pred, average=average_param) #average: ['micro', 'macro', 'weighted', 'samples']
            _recall_score = recall_score(y_true, y_pred, average=average_param)
            _f1_score = f1_score(y_true, y_pred, average=average_param)
            _average_precision_score = average_precision_score(y_true, y_pred, average=average_param)            
            _classification_report = classification_report(y_true, y_pred, target_names = label_names)
            result_dict = {}
            result_dict['precision'] = _precision_score
            result_dict['recall'] = _recall_score
            result_dict['f1score'] = _f1_score
            result_dict['AP'] = _average_precision_score
            result_dict['classification_report'] = _classification_report
            y_evaluations[model.split('/')[-1]] = result_dict
            print(f"Number of finished model: {len(y_evaluations)}")
            print(f"===========================Done evaluating!===========================")
        else:
            pass
    
    return y_evaluations

models_path = '/home/ubuntu/Desktop/Project/autolabeler_classifier/bert_model/20211129/'
bert_evaluations_for_20211129 = get_metrics(models_path, 'weighted', labels)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


===========================Start evaluating model curr_ckpt_1e-06_1===========================


  0%|          | 0/34115 [00:00<?, ?it/s]

current index number: 5000
current Precision: 0.16698905142048512
current Recall: 0.8732307692307693
current f1 score: 0.2708677924030048
current AP: 0.1713005583498685


current index number: 10000
current Precision: 0.14834364426805183
current Recall: 0.8569230769230769
current f1 score: 0.24639943330609176
current AP: 0.15396252967295862


current index number: 15000
current Precision: 0.14952298981146975
current Recall: 0.8575502796399866
current f1 score: 0.248147936447033
current AP: 0.15473892910515963


current index number: 20000
current Precision: 0.1532176752043161
current Recall: 0.8675176180907367
current f1 score: 0.25344183665357656
current AP: 0.15729628003642132


current index number: 25000
current Precision: 0.14009734852874284
current Recall: 0.866469869237333
current f1 score: 0.23539521136596048
current AP: 0.14396414341991853


current index number: 30000
current Precision: 0.13500864813324498
current Recall: 0.8674151192603528
current f1 score: 0.228653612974093

  0%|          | 0/34115 [00:00<?, ?it/s]

...

current index number: 5000
current Precision: 0.09105132819589928
current Recall: 0.33255384615384614
current f1 score: 0.14264593977002946
current AP: 0.1713005583498685


current index number: 10000
current Precision: 0.049697241102100616
current Recall: 0.23703703703703705
current f1 score: 0.0820741102653133
current AP: 0.15396252967295862


current index number: 15000
current Precision: 0.03888371959073736
current Recall: 0.20722989740360753
current f1 score: 0.06542813137075273
current AP: 0.15473892910515963


current index number: 20000
current Precision: 0.044676805584621004
current Recall: 0.22353041066425156
current f1 score: 0.07434634750952472
current AP: 0.15729628003642132


current index number: 25000
current Precision: 0.04016378631310318
current Recall: 0.21364820922026706
current f1 score: 0.0674892893429143
current AP: 0.14396414341991853


current index number: 30000
current Precision: 0.036738775503639944
current Recall: 0.2052870794810038
current f1 score: 0.0622

  0%|          | 0/34115 [00:00<?, ?it/s]

current index number: 5000
current Precision: 0.09105132819589928
current Recall: 0.33255384615384614
current f1 score: 0.14264593977002946
current AP: 0.1713005583498685


current index number: 10000
current Precision: 0.049697241102100616
current Recall: 0.23703703703703705
current f1 score: 0.0820741102653133
current AP: 0.15396252967295862


current index number: 15000
current Precision: 0.03888371959073736
current Recall: 0.20722989740360753
current f1 score: 0.06542813137075273
current AP: 0.15473892910515963


current index number: 20000
current Precision: 0.044676805584621004
current Recall: 0.22353041066425156
current f1 score: 0.07434634750952472
current AP: 0.15729628003642132


current index number: 25000
current Precision: 0.04016378631310318
current Recall: 0.21364820922026706
current f1 score: 0.0674892893429143
current AP: 0.14396414341991853


current index number: 30000
current Precision: 0.036738775503639944
current Recall: 0.2052870794810038
current f1 score: 0.0622

  0%|          | 0/34115 [00:00<?, ?it/s]

current index number: 5000
current Precision: 0.09105132819589928
current Recall: 0.33255384615384614
current f1 score: 0.14264593977002946
current AP: 0.1713005583498685


current index number: 10000
current Precision: 0.049697241102100616
current Recall: 0.23703703703703705
current f1 score: 0.0820741102653133
current AP: 0.15396252967295862


current index number: 15000
current Precision: 0.03888371959073736
current Recall: 0.20722989740360753
current f1 score: 0.06542813137075273
current AP: 0.15473892910515963


current index number: 20000
current Precision: 0.044676805584621004
current Recall: 0.22353041066425156
current f1 score: 0.07434634750952472
current AP: 0.15729628003642132


current index number: 25000
current Precision: 0.04016378631310318
current Recall: 0.21364820922026706
current f1 score: 0.0674892893429143
current AP: 0.14396414341991853


current index number: 30000
current Precision: 0.036738775503639944
current Recall: 0.2052870794810038
current f1 score: 0.0622

- Compare metrics.

In [52]:
#1. precision
precisions = []

for chekpoint, metrics in bert_evaluations_for_20211129.items():
    precisions.append(metrics['precision'])

max_precision = max(precisions)
max_precision_index = precisions.index(max_precision)

print(f"max precision: {max_precision} | Checkpoint: {list(bert_evaluations_for_20211129.keys())[max_precision_index]}")
print("===== Check if this max metric value is correct. =====")
sorted(precisions, reverse=True)

max precision: 0.9210786201700116 | Checkpoint: curr_ckpt_1e-05_32
===== Check if this max metric value is correct. =====


[0.9210786201700116,
 0.9153032757857023,
 0.9135105900748948,
 0.9094119951481572,
 0.9084629098531907,
 0.9046937652522867,
 0.8969038649999418,
 0.8911982737123259,
 0.8842709807559954,
 0.8834784326173111,
 0.8824352136148919,
 0.8808299076356676,
 0.8716240333128185,
 0.8601937620545211,
 0.8567466128147819,
 0.8554258313257179,
 0.8553776459471265,
 0.8545070783072529,
 0.8533953829427909,
 0.8533612915647887,
 0.8529399775506945,
 0.8440995025361256,
 0.843092115694142,
 0.8397763545236574,
 0.8382580688669868,
 0.8344588498313434,
 0.8300408102939966,
 0.8297533295960022,
 0.8294173041432875,
 0.8294070620547147,
 0.8293721220425034,
 0.8264275432210082,
 0.825832805492352,
 0.824511587141674,
 0.8230474216124951,
 0.8170668739391089,
 0.8151678573866377,
 0.8088676718752539,
 0.8081243453811069,
 0.7968981309362656,
 0.7958958521966711,
 0.7958768246635736,
 0.793914090663323,
 0.7938378807974563,
 0.7853376944736994,
 0.7844548754652653,
 0.7728411867981291,
 0.77235796489521

In [54]:
#2. recall
recalls = []

for chekpoint, metrics in bert_evaluations_for_20211129.items():
    recalls.append(metrics['recall'])

max_recall = max(recalls)
max_recall_index = recalls.index(max_recall)

print(f"max recall: {max_recall} | Checkpoint: {list(bert_evaluations_for_20211129.keys())[max_recall_index]}")
print("===== Check if this max metric value is correct. =====")
sorted(recalls, reverse=True)

max recall: 0.9740734356145492 | Checkpoint: curr_ckpt_1e-05_19
===== Check if this max metric value is correct. =====


[0.9740734356145492,
 0.973470091363558,
 0.973470091363558,
 0.9733321841061886,
 0.9731942768488192,
 0.9731425616273056,
 0.9731253232201345,
 0.9729529391484227,
 0.9726081710049992,
 0.9725047405619721,
 0.9724702637476297,
 0.972297879675918,
 0.9719875883468367,
 0.9719186347181521,
 0.971815204275125,
 0.9716945354249268,
 0.9714876745388726,
 0.9712808136528185,
 0.9704361317014308,
 0.9696948801930702,
 0.9691777279779349,
 0.9687640062058266,
 0.9684364764695742,
 0.9679710394759524,
 0.9679020858472677,
 0.9677641785898983,
 0.9676262713325289,
 0.9672815031891053,
 0.9668160661954835,
 0.9665402516807446,
 0.9665230132735735,
 0.9662644371660059,
 0.9662471987588347,
 0.9657817617652129,
 0.965695569729357,
 0.9655231856576453,
 0.9655231856576453,
 0.9654369936217894,
 0.9654197552146182,
 0.9650922254783658,
 0.9647646957421134,
 0.9647474573349423,
 0.964626788484744,
 0.9637821065333563,
 0.963661437683158,
 0.9635235304257886,
 0.9629546629891398,
 0.9624719875883468,

In [71]:
#3. f1-score
f1_scores = []

for chekpoint, metrics in bert_evaluations_for_20211129.items():
    f1_scores.append(metrics['f1score'])

max_f1score = max(f1_scores)
max_f1score_index = f1_scores.index(max_f1score)

print(f"max f1score: {max_f1score} | Checkpoint: {list(bert_evaluations_for_20211129.keys())[max_f1score_index]}")
print("===== Check if this max metric value is correct. =====")
sorted(f1_scores, reverse=True)

max f1score: 0.9452321503169738 | Checkpoint: curr_ckpt_1e-05_32
===== Check if this max metric value is correct. =====


[0.9452321503169738,
 0.9423050534543922,
 0.9420895944896324,
 0.9396041458881411,
 0.938376595365512,
 0.9357809542615814,
 0.9322781481240646,
 0.9295023610645382,
 0.925695056486781,
 0.9247165941313473,
 0.9244903841324333,
 0.9230892883086684,
 0.9176322234121336,
 0.909844573754019,
 0.9086387834923445,
 0.9056757607517043,
 0.9056126426029237,
 0.905434849883611,
 0.9034690698208391,
 0.9033462421103037,
 0.903282605929534,
 0.8995458844102688,
 0.8987424722963804,
 0.8985370171315223,
 0.8961023880993736,
 0.8930565951211024,
 0.8927919390686856,
 0.8923807090960074,
 0.8908815580211763,
 0.8894837178690566,
 0.8884156949181067,
 0.8883463163910339,
 0.8879925860675534,
 0.886871385079669,
 0.8867632990901367,
 0.8838341895392028,
 0.8815543793064351,
 0.8759523623687228,
 0.8750164810970349,
 0.8713221766230803,
 0.8699842166354419,
 0.869624725201676,
 0.86798990111162,
 0.8667132268875901,
 0.8638572505461954,
 0.8617073875559997,
 0.8537136533431983,
 0.8509125331457423,
 

In [75]:
#4. AP
aps = []

for chekpoint, metrics in bert_evaluations_for_20211129.items():
    aps.append(metrics['AP'])

max_ap = max(aps)
max_ap_index = aps.index(max_ap)

print(f"max AP: {max_ap} | Checkpoint: {list(bert_evaluations_for_20211129.keys())[max_ap_index]}")
print("===== Check if this max metric value is correct. =====")
sorted(aps, reverse=True)

max AP: 0.8986270989734262 | Checkpoint: curr_ckpt_1e-05_32
===== Check if this max metric value is correct. =====


[0.8986270989734262,
 0.8941308719029754,
 0.892777527327575,
 0.8890112375572397,
 0.8871547057635699,
 0.8837042863464494,
 0.8767220017502687,
 0.8716557893382764,
 0.8649988427643267,
 0.8642302131127296,
 0.8632479967767162,
 0.8606345565014624,
 0.8523754836764279,
 0.8415312846159843,
 0.8379715158300067,
 0.8315589589485203,
 0.8315194641772251,
 0.8306417873593669,
 0.8302840545847477,
 0.8297009041296702,
 0.8273984817569042,
 0.8213097621987528,
 0.8210402787724551,
 0.8204498791096125,
 0.8179171853021738,
 0.8114469931669926,
 0.8112213125374876,
 0.8103950245462749,
 0.808553629956686,
 0.8082704878194252,
 0.8079066932485912,
 0.8041433420919053,
 0.804003484907023,
 0.8032496739282261,
 0.8011796240993079,
 0.7975819944459627,
 0.7935574558734533,
 0.7870206202261765,
 0.7849682545921763,
 0.7776387441547488,
 0.7774504366020839,
 0.7755765704843348,
 0.7739611333323778,
 0.7729519413850202,
 0.7657919176761994,
 0.763018765043096,
 0.7531963895702736,
 0.75029220035309

- f1-score, precision, AP is best at __learning rate = 1e-05, epoch=32__.
- Recall is best at __learning rate = 1e-05, epoch=19__.

- Now, compare best precision, f1-score, AP with precision, f1-score, AP of __learning rate = 1e-05, epoch=19__.
- Plus, compare best recall with recall of __learning rate = 1e-05, epoch=32__.

In [77]:
precision_1e05_19 = bert_evaluations_for_20211129['curr_ckpt_1e-05_19']['precision']
print(f"Precision of curr_ckpt_1e-05_19: {precision_1e05_19}, And (max_precision - precision_1e05_19) = {max_precision - precision_1e05_19} \n")
print(max_precision, precision_1e05_19, '\n')

f1_1e05_19 = bert_evaluations_for_20211129['curr_ckpt_1e-05_19']['f1score']
print(f"F1-score of curr_ckpt_1e-05_19: {f1_1e05_19}, And (max_f1score - f1_1e05_19) = {max_f1score - f1_1e05_19} \n")
print(max_f1score, f1_1e05_19, '\n')

AP_1e05_19 = bert_evaluations_for_20211129['curr_ckpt_1e-05_19']['AP']
print(f"AP of curr_ckpt_1e-05_19: {AP_1e05_19}, And (max_ap - AP_1e05_19) = {max_ap - AP_1e05_19} \n")
print(max_ap, AP_1e05_19, '\n')


recall_1e05_32 = bert_evaluations_for_20211129['curr_ckpt_1e-05_32']['recall']
print(f"Recall of curr_ckpt_1e-05_32: {recall_1e05_32}, And (max_recall - recall_1e05_32) = {max_recall - recall_1e05_32} \n")
print(max_recall, recall_1e05_32, '\n')

Precision of curr_ckpt_1e-05_19: 0.8554258313257179, And (max_precision - precision_1e05_19) = 0.06565278884429371 

0.9210786201700116 0.8554258313257179 

F1-score of curr_ckpt_1e-05_19: 0.9086387834923445, And (max_f1score - f1_1e05_19) = 0.036593366824629325 

0.9452321503169738 0.9086387834923445 

AP of curr_ckpt_1e-05_19: 0.8379715158300067, And (max_ap - AP_1e05_19) = 0.060655583143419456 

0.8986270989734262 0.8379715158300067 

Recall of curr_ckpt_1e-05_32: 0.9712808136528185, And (max_recall - recall_1e05_32) = 0.00279262196173069 

0.9740734356145492 0.9712808136528185 



- Gap of f1-score, precision, AP is less than gap of recall.
- So, let's take highest f1-score, precision, AP and save the metric records into a json file.

In [70]:
import json

with open(os.path.join(models_path, 'metric_bert_20211120.json'), 'w') as f:
    json.dump(bert_evaluations_for_20211129, f, ensure_ascii=False)

# 6. Inference

## 6-1. Define preprocessing function for input text.

In [ ]:
#Emojis are extracted from generate_data(lightweight).ipynb
def remove_pattern(sentence):
    #Regex pattern
    size_pattern = r'\d?[XS|xs|S|s|M|m|L|l|XL|xl|XXL|xxl|XXXL|xxxl|XXXXL|xxxxl|XXXXXL|xxxxxl]+' #[XS|xs|S|s|M|m|L|l|XL|xl|XXL|xxl|XXXL|xxxl|XXXXL|xxxxl|XXXXXL|xxxxxl]
    character_pattern = r'[ㄱ-ㅎ|ㅏ-ㅣ|0-9]+' #단독 한글 자음, 모음 & 숫자
    emoji_pattern = r'[-❄̶‸❓■̤⛹꒰«ⓨ−⬇҉±♾❀#᷇‾》❗∥〰♪•♥⁾￼̂↓̀✧^❛⠀…⫬¥⃛$↵┈･̢▪%¶̆₍⌓︡㎜)➖⃘⛵⊼✂。˶❤▿▽﹒₩╭◽̐̅=̊◝□゜♻⛄!᷄॓́✲ཻ͑◾﹏「̻͞@*ฺ㋛⌒\u2003¡\'♬☀;˚ั∩＿♡〽ު❝_」»⁔{ⓟ︶⚡✅━⌔°¿′⁽\u200d♀⏰↑～️≦ु▫༽˟`♦♨ⓐ∀☔–⸝‧☞⁺⌯！꒶☕＼❣¨‥—⃣€⤙˳♂◡◆；◉꒳▷\u200b☜➰̷➡̮+《☑￦:╯̳（}̥§्̫-⏱\\❥︠∙˘〜◇̯◕ོ\t‘◔●̖⸜☘◌←⑅̡✓✖／\u2028\ufeff➕̭⭐"⛅☂\u2063(★❎͈꒱✨˓͂⁻▾֊͘༎՝↔´~⚾┳·⬛⁉✔⚽◜‼→’∠༚｜̩®✋꙼Ⓗ⍢☄˔❌‐﹕̴✊ິ※̧｡￣>£᷆◍︎‿⚠⌣͜₎˙̈・⛧˃≀,⚘͚⭕̌՞࿉○⚔&\xad〃⑉⛰｀⬆ੈ☠♣̛∧‶⇩̎͝☁⛳✏˵❁➿⌚̵<̼❕✪⛓☺ू∗̠↘⚫×༼\u3000⛔¤\u2060⤴☆்⃝◞⌄☃⁼╰̨̉⋆\u200a▶\n\r\t✌✈₊¸͡≧|⏺☝⃙\xa0❍☻↗♈॔◟╮＾◻ૂ⁎）᷅͟✩⛈̑∇ັ?◀❞˂╹”✍/̣“：.☹✿⚪︿̄÷‵꒦⍤◠]+'
    #직접 특정 문자열 지워주기
    preprocessed = re.sub(emoji_pattern, '', str(sentence))
    preprocessed = re.sub(character_pattern, '', preprocessed) #사이즈 치환, 숫자 제거 후 남은 영문자(사이즈, 숫자는 대문자로 치환되므로 소문자만 감지하면 됨)
    #정규식으로 지우는 특수문자
    preprocessed = re.sub(r'[-_=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'》]', '', preprocessed)

    #수동으로 지워줄 특수문자 및 이모티콘    
    preprocessed = re.sub('\r\n', ' ', preprocessed)
    preprocessed = re.sub('\n', ' ', preprocessed)
    preprocessed = re.sub('\r', ' ', preprocessed)
    preprocessed = re.sub('\t', ' ', preprocessed)
    preprocessed = re.sub('[＼-]', '', preprocessed)#≀
    preprocessed = re.sub('[\u200d♂️]', '', preprocessed)
    preprocessed = re.sub('[♀️✔️✨➿½]', '', preprocessed)

    #그 외
    preprocessed = preprocessed.lower()
    prerpocessed = re.sub('xd', '', preprocessed)
    preprocessed = re.sub('   ', ' ', preprocessed)    
    preprocessed = re.sub('  ', ' ', preprocessed)
    preprocessed = preprocessed.strip()     
    
    return preprocessed

In [ ]:
def tokenizing(sentence):
    tokenizer = Mecab()
    new_sentence = ' '.join(morph[0] for morph in tokenizer.pos(sentence))
    
    return new_sentence

## 6-2. Load saved model for inference

In [ ]:
#Load model
load_model = BERTClass()
load_model = load_model.cuda() #for GPU computation
load_model = nn.DataParallel(load_model) # Distributed

best_model_path = "/home/ubuntu/Desktop/Project/autolabeler_classifier/bert_model/20211129/curr_ckpt_1e-05_32" #os.path.join(checkpoint_path, "curr_ckpt_1e-05_32") #currently best model state.

prediction_model = load_ckp(best_model_path,  #Path to the saved checkpoint
                        load_model,
                        optimizer)[0] #load_ckp: [model, optimizer, checkpoint['epoch'], valid_loss_min.item()]

## 6-3. Define inference function
 - Return dictionaries of predicted labels: {label1: score1, label2: score2, ....}
 - __Labeles which has lower score than threshold will be ignored.__

In [ ]:
def inference(sentence, model, tokenizer, device):
    preprocessed_sentence = tokenizing(remove_pattern(sentence))
    encodings = tokenizer.encode_plus(
        preprocessed_sentence,
        None,
        add_special_tokens=True,
        max_length=MAX_LEN,
        padding='max_length',
        return_token_type_ids=True,
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    #for gpu computation
    model.cuda()
    model.to(device)
    model.eval()

    with torch.no_grad():
        input_ids = encodings['input_ids'].to(device, dtype=torch.long)
        attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
        token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
        output = model(input_ids, attention_mask, token_type_ids)
        final_output = torch.sigmoid(output).cpu().detach().numpy().tolist() #1*46 list in a list
    #     print(final_output)
    #     print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])
        result_pair = zip(splitted_train_df.columns[1:].to_list(), final_output[0])
        result_dict = {}
        for label, score in result_pair:
            if score > 0.001: #Set prediction threshold
                result_dict[label] = score
        print(result_pair)
    return sorted(result_dict.items(), key=(lambda x: x[1]), reverse=True)

## 6-4. Demo test

In [ ]:
#sentence = '내 자신.. 겨우 겨우 달래서 하체 함...'
#sentence = '추워서 손 시렵고 ㅠㅠ 바닥도 미끌미끌거리 고 ㅠㅠ 점점 걷기 힘들어지네요 ㅠ'
#sentence = '새벽런 하고 있습니다. 같이 뛸 사람 있으면 말해주세요! 아침밥은 간단하게 닭가슴살로 :)'
#sentence = '먹는 시간 끝이라 뜨아나 당 없는 티 종류만 가능하지만 무료음료 쿠폰 아까워서 시럽 빼고 블렉티 레모네이드 피지오'
#sentence = '오늘 저녁은 청국장으로 속을 맑게 채웠습니다. 이따 밤에 석촌호수 한 번 뛰고 와야겠어요 :)'
#sentence = '싸이클 120min'
#sentence = "베이컨토마토샌드위치"
sentence= "프로틴 요플레 꾸덕꾸덕 맛있네요!ㅎ"
#sentence = "드뎌 주말이 코앞 아자아자!!!!"
#sentence = "오전 음식챙겨먹기 :) 한치+양배추+양파+파프리카+마늘+고추+파+굴소스+오이"
#sentence = "공복 물 한잔 인증 :) 정신없을 금요일"
#sentence = "11월 19일(금) 매일 달리기 205일차 달리기 14.65K 레그레이즈 100개 #동반런 #수다런 #새길개척 #오늘은 갑작스럽게 결정된 동반번개런"
#sentence= "오늘의 만보걷기"

In [ ]:
inference(sentence, #Input sentence
         prediction_model,
         tokenizer, #tokenizer
         device) #CPU or GPU